In [ ]:
!conda install -y py2neo

In [ ]:
from py2neo import Graph, Node, Relationship, authenticate

*Important Note* - when calling the authenticate function, pass in the password you used during installation as the last argument. In this exercise file, "neo4j_train" is the password. Your's may be different.

In [ ]:
authenticate("localhost", "neo4j", "neo4j_train")

In [ ]:
g = Graph(host='localhost')#create a graph database

In [ ]:
type(g)

In [ ]:
!ls *.txt

In [ ]:
!head node_city.txt

In [ ]:
#this is a transaction which needs to have a begin and commit an between those 2 is a transaction
# this transaction is used to create the node city
tx = g.begin()
with open('node_city.txt') as f_in:
    for line in f_in:
        city_list = line.rstrip().split(',')
        city = Node("City",name=city_list[0], population=city_list[1])
        tx.create(city)
tx.commit()

In [ ]:
g.data("MATCH (c:City) RETURN c.name, c.population")#equivalent to sql command select 
#all nodes of type city are matched and it returns 2 properties

In [ ]:
g.data("MATCH (c:City) RETURN c.name, c.population LIMIT 10")

In [ ]:
#select nodes which are of type cities
result_set = g.node_selector.select("City")

In [ ]:
type(result_set)

In [ ]:
result_set.first()

In [ ]:
LA = result_set.where(name="Los_Angeles")

In [ ]:
type(LA)

In [ ]:
LA.first()

In [ ]:
for r in result_set:
    print r

In [ ]:
!ls *.txt

In [ ]:
!head edge_distance.txt

In [ ]:
tx = g.begin()
with open('edge_distance.txt') as f_in:
    for line in f_in:
        edge_list = line.rstrip().split(',')
        city1_name = edge_list[0]
        city2_name = edge_list[1]
        city1_node = result_set.where(name=city1_name)
        city2_node = result_set.where(name=city2_name)
        city_pair = Relationship(city1_node, "FLIGHT_BETWEEN", city2_node)
        city_pair["distance"] = edge_list[2]
        tx.create(city_pair)
tx.commit()

In [ ]:
g.relationship_types

In [ ]:
import pandas as pd

In [ ]:
flight_list = list()

In [ ]:
for cities in g.match(rel_type="FLIGHT_BETWEEN"):
    city1_name = cities.nodes()[0]['name']
    city2_name = cities.nodes()[1]['name']
    distance = cities['distance']
    flight_list.append([city1_name, city2_name, distance])

In [ ]:
type(flight_list)

In [ ]:
flight_list[0]

In [ ]:
flight_df = pd.DataFrame(flight_list)

In [ ]:
flight_df